In [4]:
import fundamentalanalysis as fa
import financedatabase as fd
import yfinance as yf

import pandas as pd
import datetime as dt
from pathlib import Path
from os.path import exists

In [5]:
pd.set_option('display.max_columns', None)  

In [6]:
api_key = "c3f92edf0e79c46818aabbb8dc76043f"

day1 = '2023-01-31'
month1 = str(day1)[:7]
year1 = str(day1)[:4]

save_hist = Path(f"/home/gdp/hot_box/etfs/data/hist/{year1}/{month1}/{day1}/")
if not save_hist.exists():
    save_hist.mkdir(parents=True)

save_profile = Path(f"/home/gdp/hot_box/etfs/data/profile/")
if not save_profile.exists():
    save_profile.mkdir(parents=True)

save_bulk = Path(f"/home/gdp/hot_box/etfs/data/bulk/{year1}/{month1}/{day1}/")
if not save_bulk.exists():
    save_bulk.mkdir(parents=True)    

In [7]:
# Show the available companies
companies = fa.available_companies(api_key)
companies = companies.drop_duplicates()
print(companies.shape)

companies.to_csv(save_bulk / "bulk_tickers.csv")

(66943, 5)


In [8]:
etfs = pd.DataFrame(companies[companies['type'] == 'etf'])
etfs = etfs.drop_duplicates()
etfs.to_csv(save_bulk / "etf_tickers.csv")
etfs

,name,price,exchange,exchangeShortName,type
symbol,,,,,
GDIV,Harbor Dividend Growth Leaders ETF,12.280,New York Stock Exchange Arca,AMEX,etf
BOND,PIMCO Active Bond Exchange-Traded Fund,94.200,New York Stock Exchange Arca,AMEX,etf
TSL,GraniteShares ETF Trust - GraniteShares 2x Lon...,12.960,NASDAQ Global Market,NASDAQ,etf
FTXN,First Trust Nasdaq Oil & Gas ETF,28.260,NASDAQ Global Market,NASDAQ,etf
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,57.460,NASDAQ Global Market,NASDAQ,etf
...,...,...,...,...,...
REC,Emles Real Estate Credit ETF,20.055,BATS,ETF,etf
TCBT.L,VanEck iBoxx EUR Corporates UCITS ETF,16.952,London Stock Exchange,LSE,etf
HEDF.L,WisdomTree Europe Equity UCITS ETF,19.350,London Stock Exchange,LSE,etf


In [9]:
exchange_list = [
    # 'MCX',
    'Other OTC',
    'Toronto Stock Exchange',
    # 'Saudi',
    'New York Stock Exchange Arca',
    'Hamburg',
    'Amsterdam',
    'KSE',
    'Munich',
    'New York Stock Exchange',
    'SES',
    'Taiwan',
    'Milan',
    'BATS',
    'Mexico',
    'London Stock Exchange',
    'Nasdaq Global Select',
    'NASDAQ Capital Market',
    'Swiss Exchange',
    'Paris',
    'NASDAQ Global Select',
    'Frankfurt',
    'NEO',
    'Brussels',
    'NASDAQ Global Market',
    'American Stock Exchange',
    # 'XETRA'
]

In [10]:
etfs[etfs['exchange'].isin(exchange_list)]

,name,price,exchange,exchangeShortName,type
symbol,,,,,
GDIV,Harbor Dividend Growth Leaders ETF,12.280,New York Stock Exchange Arca,AMEX,etf
BOND,PIMCO Active Bond Exchange-Traded Fund,94.200,New York Stock Exchange Arca,AMEX,etf
TSL,GraniteShares ETF Trust - GraniteShares 2x Lon...,12.960,NASDAQ Global Market,NASDAQ,etf
FTXN,First Trust Nasdaq Oil & Gas ETF,28.260,NASDAQ Global Market,NASDAQ,etf
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,57.460,NASDAQ Global Market,NASDAQ,etf
...,...,...,...,...,...
REC,Emles Real Estate Credit ETF,20.055,BATS,ETF,etf
TCBT.L,VanEck iBoxx EUR Corporates UCITS ETF,16.952,London Stock Exchange,LSE,etf
HEDF.L,WisdomTree Europe Equity UCITS ETF,19.350,London Stock Exchange,LSE,etf


In [11]:
list(set(etfs['exchange']))

['Toronto Stock Exchange',
 'Other OTC',
 'London Stock Exchange',
 'Saudi',
 'New York Stock Exchange Arca',
 'Brussels',
 'Mexico',
 'New York Stock Exchange',
 'Frankfurt',
 'KSE',
 'Swiss Exchange',
 'SES',
 'NASDAQ Global Market',
 'Hamburg',
 'XETRA',
 'NASDAQ Capital Market',
 'American Stock Exchange',
 'Paris',
 'NEO',
 'MCX',
 'NASDAQ Global Select',
 'Milan',
 'Munich',
 'Amsterdam',
 'Nasdaq Global Select',
 'BATS',
 'Taiwan']

In [12]:
def source_profiles(ticker_list):
    good_etf_tickers = []
    bad_etf_tickers = []
    
    try:
        for ticker in ticker_list:
            x = pd.DataFrame(fa.profile(ticker, api_key))
            x.columns = [ticker]
            x.to_csv(save_profile / f"{ticker}_profile.csv")
            good_etf_tickers.append(ticker)

    except Exception as e:
        print(f"X X X ERROR X X X \n [ X ] {ticker} - {e}")
        bad_etf_tickers.append(ticker)
        pass

    return sorted(good_etf_tickers), sorted(bad_etf_tickers)

In [13]:
etf_ticker_list = list(etfs.index)


# good_etf_tickers, bad_etf_tickers = source_profiles(etf_ticker_list)

In [14]:
core_selection = fd.select_etfs(exclude_exchanges=True)

tickers = pd.Series(core_selection.keys())
tickers.to_excel(save_bulk / f'core_selection_tickers.xlsx', index=None, header=None)

In [15]:
all_etfs_df = pd.DataFrame.from_dict(fd.select_etfs(), orient='index').reset_index()
all_etfs_df.head()

,index,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
0,AAA,Listed Funds Trust AAF First Pr,AAF First Priority CLO Bond ETF,USD,The investment seeks capital preservation and ...,None,Alternative Access Funds,PCX,us_market,1.001962e+07
1,AAAU,Goldman Sachs Physical Gold ETF,Goldman Sachs Physical Gold ETF,USD,The investment seeks to provide investors with...,None,Exchange Traded Concepts,PCX,us_market,3.317956e+08
2,AADR,AdvisorShares Dorsey Wright,AdvisorShares Dorsey Wright ADR ETF,USD,None,None,None,NMS,us_market,NaN
3,AAXJ,iShares MSCI All Country Asia e,iShares MSCI All Country Asia ex Japan ETF,USD,The investment seeks to track the investment r...,Pacific/Asia ex-Japan Stk,iShares,NMS,us_market,6.588896e+09
4,ABCS,Guggenheim ABC High Dividend ET,None,None,None,None,None,PCX,us_market,NaN


In [16]:
gas = pd.DataFrame.from_dict(fd.search_products(core_selection, 'gas'), orient='index')
gas.head()

,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
BOIL,ProShares Ultra Bloomberg Natur,ProShares Ultra Bloomberg Natural Gas,USD,"The investment seeks daily investment results,...",Trading--Leveraged Commodities,ProShares,PCX,us_market,9.201114e+07
CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07
DDG,ProShares Short Oil & Gas,ProShares Short Oil & Gas,USD,The investment seeks daily investment results ...,Trading--Inverse Equity,ProShares,PCX,us_market,3.326665e+06


In [17]:
oil = pd.DataFrame.from_dict(fd.search_products(core_selection, 'oil'), orient='index')
oil.head()

,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07
DBO,Invesco DB Oil Fund,Invesco DB Oil Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,5.195419e+08


In [18]:
dodds_etf_df = oil.append(gas).reset_index().drop_duplicates().sort_values('index')
dodds_etf_df.columns = ['ticker', 'short_name', 'long_name', 'currency', 'summary', 'category', 'family', 'exchange', 'market', 'total_assets']
dodds_etf_df.head()

,ticker,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
0,BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
37,BOIL,ProShares Ultra Bloomberg Natur,ProShares Ultra Bloomberg Natural Gas,USD,"The investment seeks daily investment results,...",Trading--Leveraged Commodities,ProShares,PCX,us_market,9.201114e+07
1,CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
2,DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
3,DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07


In [19]:
x = fd.select_etfs()
x1 = pd.DataFrame.from_dict(x, orient='index')
y = pd.DataFrame.from_dict(fd.search_products(database=x, query='oil', search='summary'), orient='index')
y.head()

,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
CRAK,VanEck Vectors Oil Refiners ETF,VanEck Vectors Oil Refiners ETF,USD,The investment seeks to replicate as closely a...,Equity Energy,VanEck,PCX,us_market,2.035487e+07
DBC,Invesco DB Commodity Index Trac,Invesco DB Commodity Index Tracking Fund,USD,"The investment seeks to track changes, whether...",Commodities Broad Basket,Invesco,PCX,us_market,1.891862e+09
DBE,Invesco DB Energy Fund,Invesco DB Energy Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,8.291202e+07
DBO,Invesco DB Oil Fund,Invesco DB Oil Fund,USD,The investment seeks to track the DBIQ Optimum...,None,Invesco,PCX,us_market,5.195419e+08


In [20]:
category_list = [
    # 'Short Government',
    # 'Convertibles',
    # 'Infrastructure',
    # 'Foreign Large Blend',
    # 'Foreign Small/Mid Blend',
    # 'Miscellaneous Sector',
    # 'Foreign Large Value',
    # 'Muni California Long',
    # 'Mid-Cap Growth',
    # 'Financial',
    # 'Short-Term Bond',
    # 'India Equity',
    # 'Foreign Large Growth',
    'Equity Energy',
    # 'Muni Minnesota',
    # 'Latin America Stock',
    # 'Mid-Cap Blend',
    # 'Long-Short Equity',
    # 'Miscellaneous Region',
    # 'Allocation--85%+ Equity',
    # 'Small Blend',
    # 'China Region',
    # 'Intermediate Government',
    # 'Single Currency',
    # 'Multialternative',
    # 'Large Growth',
    'Energy Limited Partnership',
    # 'High Yield Bond',
    # 'Consumer Defensive',
    # 'Volatility',
    'Trading--Inverse Commodities',
    # 'Market Neutral',
    # 'Diversified Pacific/Asia',
    # 'Communications',
    # 'Health',
    # 'Trading--Leveraged Debt',
    # 'Foreign Small/Mid Value',
    # 'Pacific/Asia ex-Japan Stk',
    # 'Muni National Interm',
    # 'Managed Futures',
    # 'Muni National Short',
    # 'Bear Market',
    'Natural Resources',
    # None,
    # 'Consumer Cyclical',
    # 'Small Growth',
    # 'High Yield Muni',
    'Trading--Leveraged Commodities',
    # 'Long-Short Credit',
    # 'Ultrashort Bond',
    # 'Option Writing',
    # 'Equity Precious Metals',
    # 'Allocation--15% to 30% Equity',
    'Utilities',
    # 'Preferred Stock',
    # 'Long Government',
    # 'Corporate Bond',
    'Commodities Broad Basket',
    # 'Multisector Bond',
    # 'Small Value',
    # 'World Bond',
    # 'Bank Loan',
    # 'Allocation--70% to 85% Equity',
    # 'Japan Stock',
    # 'Tactical Allocation',
    # 'Multicurrency',
    # 'Nontraditional Bond',
    # 'Long-Term Bond',
    # 'Large Blend',
    # 'World Allocation',
    # 'Foreign Small/Mid Growth',
    # 'Large Value',
    'Industrials',
    # 'Global Real Estate',
    # 'Muni National Long',
    # 'Europe Stock',
    # 'World Stock',
    # 'Trading--Inverse Equity',
    # 'Trading--Leveraged Equity',
    # 'Diversified Emerging Mkts',
    # 'Inflation-Protected Bond',
    # 'Allocation--50% to 70% Equity',
    # 'Trading--Inverse Debt',
    # 'Technology',
    # 'Allocation--30% to 50% Equity',
    # 'Intermediate-Term Bond',
    # 'Trading--Miscellaneous',
    # 'Real Estate',
    # 'Mid-Cap Value',
    # 'Emerging Markets Bond',
    # 'Muni New York Intermediate',
    # 'Emerging-Markets Local-Currency Bond'
 ]

In [21]:
for c in category_list:
    z = pd.DataFrame.from_dict(fd.search_products(database=x, query=c, search='summary'), orient='index')
    y = y.append(z)

In [22]:
y = pd.DataFrame(y).reset_index().sort_values('index')
yy = y.drop_duplicates(subset=['index'])
yy = yy.rename(columns={"index": "ticker"})
# yy.to_csv(save_bulk)

In [23]:
dodds_etf_df = pd.DataFrame(dodds_etf_df.append(yy).drop_duplicates(subset=['ticker'])).sort_values('ticker')
dodds_etf_df

,ticker,short_name,long_name,currency,summary,category,family,exchange,market,total_assets
66,ARMR,Exchange Listed Funds Trust Arm,Armor US Equity Index ETF,USD,The investment seeks to provide investment res...,Large Blend,Exchange Traded Concepts,PCX,us_market,6.117832e+06
0,BNO,"United States Brent Oil Fund, L","United States Brent Oil Fund, LP",USD,The investment seeks the daily changes in perc...,None,USCF Investments,PCX,us_market,3.183956e+08
37,BOIL,ProShares Ultra Bloomberg Natur,ProShares Ultra Bloomberg Natural Gas,USD,"The investment seeks daily investment results,...",Trading--Leveraged Commodities,ProShares,PCX,us_market,9.201114e+07
67,CHII,Global X MSCI China Industrials,Global X MSCI China Industrials ETF,USD,The investment seeks to provide investment res...,China Region,Global X Funds,PCX,us_market,2.409662e+06
48,CHIU,Global X Funds Global X MSCI Ch,Global X MSCI China Utilities ETF,USD,The investment seeks to provide investment res...,None,Global X Funds,PCX,us_market,1.706715e+06
...,...,...,...,...,...,...,...,...,...,...
64,VPU,Vanguard Utilities ETF,Vanguard Utilities Index Fund ETF Shares,USD,The investment seeks to track the performance ...,Utilities,Vanguard,PCX,us_market,6.270528e+09
34,XES,SPDR Series Trust SPDR S&P Oil,SPDR S&P Oil & Gas Equipment & Services ETF,USD,The investment seeks to provide investment res...,Equity Energy,SPDR State Street Global Advisors,PCX,us_market,1.547576e+08
35,XLE,SPDR Select Sector Fund - Energ,Energy Select Sector SPDR Fund,USD,The investment seeks to provide investment res...,Equity Energy,SPDR State Street Global Advisors,PCX,us_market,2.308107e+10
73,XLU,SPDR Select Sector Fund - Utili,Utilities Select Sector SPDR Fund,USD,The investment seeks to provide investment res...,Utilities,SPDR State Street Global Advisors,PCX,us_market,1.236013e+10


In [24]:
# dodds_etf_df.to_csv(save_bulk / f"dodds_etf_df.csv")
dodds_etf_df.to_csv(save_bulk / f"dodds_etf_df.csv")

In [25]:
dodds_etf_ticker_list = list(dodds_etf_df["ticker"])
len(dodds_etf_ticker_list)

86

In [26]:
dodds_etf_hist_df = yf.download(dodds_etf_ticker_list, period='1y', rounding=True)['Adj Close']

[*********************100%***********************]  86 of 86 completed

1 Failed download:
- FRAK: No data found for this date range, symbol may be delisted


In [27]:
dodds_etf_hist_df.dropna(axis=1)

,ARMR,BNO,BOIL,CHII,CHIU,CRAK,DBC,DBE,DBO,DIG,DRIP,DUG,DUSL,EDOW,EINC,EMLP,ERTH,ERX,ERY,ETHO,EXI,FCG,FIDU,FTRI,FTXN,FUE,FUTY,FXR,FXU,GAZ,GNR,GUNR,GUSH,HAP,IDU,IEO,IEZ,IGE,IYE,IYJ,JXI,KOLD,MUNI,NANR,NRGD,NRGU,OIH,OILK,PDBC,PSCE,PSCU,PUI,PXE,PXJ,RGI,RLY,RYU,SCO,SDP,SIJ,UCO,UGA,UGAZF,UMI,UNG,UNL,UPW,USL,USO,USOI,UTES,UTSL,UXI,VIS,VPU,XES,XLE,XLU,XOP
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-02-02,25.55,24.22,56.02,16.23,17.91,29.01,22.60,19.70,15.16,28.25,37.60,30.18,42.60,31.62,59.21,25.81,59.56,44.72,71.00,58.85,118.33,20.39,53.60,15.06,23.04,10.99,44.75,58.39,32.21,28.51,57.85,42.58,117.02,49.68,85.05,74.35,15.85,35.81,36.36,108.44,63.21,85.24,55.09,48.89,1795.00,306.10,232.65,46.22,15.33,8.07,63.58,34.60,23.71,3.81,187.97,28.24,112.84,48.30,13.19,14.87,28.12,48.01,13.50,32.20,18.98,16.08,69.56,31.72,62.52,106.60,45.51,35.87,29.75,193.78,151.33,60.23,68.49,69.34,111.84
2022-02-03,25.34,24.60,45.56,16.15,17.82,28.68,22.65,19.80,15.36,27.64,38.70,30.86,40.57,31.21,58.72,25.66,58.47,43.88,72.20,57.62,115.98,20.14,52.63,14.96,22.77,10.80,44.57,57.42,32.01,25.97,57.23,42.19,113.73,49.04,84.41,73.42,15.59,35.39,35.92,106.26,62.88,99.76,55.17,48.40,1840.00,297.79,229.72,46.93,15.32,7.94,63.16,34.33,23.49,3.77,184.67,28.03,112.58,46.85,13.29,15.51,28.97,48.42,9.99,31.94,17.24,15.44,69.03,32.22,63.51,107.20,45.24,35.35,28.47,190.35,150.37,59.14,67.84,69.03,110.21
2022-02-04,25.42,25.20,38.16,16.22,17.95,29.03,22.83,19.96,15.59,28.48,37.40,29.87,39.29,31.17,59.21,25.59,59.10,45.23,69.90,57.80,115.35,20.45,52.18,15.03,23.03,10.91,44.19,56.94,31.71,23.81,57.75,42.56,117.30,49.32,83.87,74.49,15.84,35.69,36.46,105.58,62.43,117.36,55.13,48.91,1745.00,314.06,234.40,47.84,15.45,8.11,63.27,34.12,23.85,3.87,182.76,28.20,111.59,45.00,13.49,15.69,30.12,49.10,7.45,32.21,15.84,14.52,68.06,32.83,64.91,107.20,44.96,34.58,28.17,188.67,149.10,60.63,68.90,68.48,111.92
2022-02-07,25.51,25.07,33.85,16.24,18.06,29.40,22.88,19.89,15.60,29.12,37.80,29.21,39.47,31.10,59.20,25.63,58.61,46.43,68.20,57.74,115.57,20.24,52.19,15.28,23.04,11.10,44.10,56.85,31.67,22.55,58.36,43.10,116.12,49.88,83.68,74.66,16.05,36.02,36.91,105.20,62.08,129.04,55.08,49.38,1675.00,326.98,236.60,47.75,15.48,7.95,63.19,34.16,23.79,3.87,182.81,28.28,111.47,45.15,13.58,15.86,30.03,49.40,6.31,32.30,14.99,13.82,67.51,32.77,64.68,108.00,44.76,34.41,27.86,188.89,148.91,60.19,69.78,68.38,111.22
2022-02-08,25.60,24.62,32.77,16.39,18.27,29.15,22.68,19.59,15.39,27.97,40.40,30.39,40.56,31.42,58.80,25.61,59.09,44.46,71.10,58.62,116.55,19.57,52.86,15.39,22.51,10.79,44.15,57.81,31.71,22.18,58.71,43.31,108.60,50.16,83.81,72.89,15.74,35.60,36.17,106.01,62.30,133.40,54.99,49.12,1775.00,305.70,231.50,46.99,15.37,7.70,64.02,34.25,22.93,3.79,185.04,28.30,111.78,46.60,13.52,15.51,29.04,48.36,6.00,32.15,14.72,13.62,67.78,32.22,63.62,107.60,44.80,34.47,28.43,191.28,149.19,58.88,68.28,68.43,107.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26,21.46,28.73,8.12,14.12,15.59,33.92,25.05,22.53,15.42,47.46,12.28,10.95,34.00,29.90,64.39,27.53,52.08,74.69,25.60,53.45,112.40,25.64,52.67,14.92,30.16,12.01,44.72,55.47,33.18,15.38,62.08,46.61,165.10,52.71,84.65,98.78,22.93,43.51,49.07,100.80,60.40,50.33,52.46,58.79,249.77,603.94,329.49,45.36,15.10,10.78,57.68,33.77,31.79,5.49,187.02,29.10,115.19,22.99,11.32,14.43,30.91,63.01,3.81,35.12,9.62,13.48,66.35,35.95,70.93,84.02,46.02,31.16,24.21,190.11,150.50,87.06,93.11,68.89,145.45
2023-01-27,21.55,28.32,8.47,14.08,15.54,33.60,24.77,22.12,15.12,45.78,12.61,11.32,34.35,29.91,64.44,27.49,52.80,71.75,26.64,53.71,112.35,25.48,52.92,14.70,29.51,12.17,44.72,55.79,33.25,15.69,61.41,46.12,160.19,52.28,84.65,97.54,22.69,43.01,48.20,101.53,60.19,48.23,52.43,58.22

In [28]:
for i in dodds_etf_ticker_list:
    print(i)

ARMR
BNO
BOIL
CHII
CHIU
CRAK
DBC
DBE
DBO
DDG
DIG
DRIP
DUG
DUSL
EDOW
EINC
EMLP
ERTH
ERX
ERY
ETHO
EXI
FCG
FIDU
FRAK
FTRI
FTXN
FUE
FUTY
FXR
FXU
GAZ
GNR
GUNR
GUSH
HAP
IDU
IEO
IEZ
IGE
IMLP
IYE
IYJ
JHMI
JHMU
JXI
KOLD
MUNI
NANR
NRGD
NRGU
OIH
OILK
PDBC
PSCE
PSCI
PSCU
PUI
PXE
PXJ
PZD
RGI
RLY
RYU
SCO
SDP
SIJ
UCO
UGA
UGAZF
UMI
UNG
UNL
UPW
USL
USO
USOI
UTES
UTSL
UXI
VIS
VPU
XES
XLE
XLU
XOP


In [29]:
for i in ['UGAZF', 'NRGU', 'GUSH', 'BOIL', 'ERX', 'DIG', 'UCO', 'PSCE', 'XES', 'FCG', 'XOP', 'UNG', 'PXE', 'GAZ', 'OIH', 'IEZ', 'PXJ', 'FTXN', 'IEO', 'DUSL', 'IYE', 'XLE', 'UNL', 'BNO', 'USO', 'OILK', 'UGA', 'IGE', 'USL', 'DBO', 'DBE', 'NANR', 'EINC', 'UXI', 'GNR']:
    print(i)

UGAZF
NRGU
GUSH
BOIL
ERX
DIG
UCO
PSCE
XES
FCG
XOP
UNG
PXE
GAZ
OIH
IEZ
PXJ
FTXN
IEO
DUSL
IYE
XLE
UNL
BNO
USO
OILK
UGA
IGE
USL
DBO
DBE
NANR
EINC
UXI
GNR


In [1]:
import pandas as pd

In [3]:
data = pd.read_pickle("/home/gdp/hot_box/i4m/data/recommenders/2023/2023-02/2023-02-01/a1_finviz.pkl")
data

,no,ticker,company,sector,industry,country,market_cap,p/e,forward_p/e,peg,...,relative_volume,volume,target_price,prev_close,price,after_hours_close,after_hours_change,over_50day_low,rev_per_ee,income_per_ee
0,1,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research,USA,46184.19,37.15,24.64,3.10,...,1.14,1499909,163.71,152.08,155.45,155.45,0.00%,1662.57,378.34,69.28
1,2,AA,Alcoa Corporation,Basic Materials,Aluminum,USA,9541.68,0.00,11.08,0.00,...,1.06,5473532,53.18,52.24,54.58,54.74,0.29%,178.89,1020.57,-8.36
2,3,AAAU,Goldman Sachs Physical Gold ETF,Financial,Exchange Traded Fund,USA,0.00,0.00,0.00,0.00,...,1.16,1235178,0.00,19.13,19.38,19.35,-0.13%,152.48,0.00,0.00
3,4,AAC,Ares Acquisition Corporation,Financial,Shell Companies,USA,1272.57,47.30,0.00,0.00,...,1.49,1316205,0.00,10.16,10.17,0.00,0.0,535.26,0.00,0.00
4,5,AACG,ATA Creativity Global,Consumer Defensive,Education & Training Services,China,48.37,0.00,0.00,0.00,...,0.55,4775,19.00,1.44,1.48,1.48,0.00%,4.78,50.68,-13.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8346,8347,ZVSA,"ZyVersa Therapeutics, Inc.",Healthcare,Biotechnology,USA,16.35,0.00,0.00,0.00,...,0.21,59455,0.00,1.84,1.76,1.80,2.25%,6.38,0.00,-333.33
8347,8348,ZWS,Zurn Elkay Water Solutions Corporation,Industrials,Pollution & Treatment Controls,USA,3989.53,51.66,23.85,3.04,...,0.82,1091638,26.67,21.86,22.11,22.11,0.00%,221.10,902.92,38.46
8348,8349,ZYME,Zymeworks Inc.,Healthcare,Biotechnology,Canada,594.39,0.00,0.00,0.00,...,0.33,397797,13.45,9.45,9.27,9.27,0.00%,15.21,104.55,-783.92
8349,8350,ZYNE,"Zynerba Pharmaceuticals, Inc.",Healthcare,Drug Manufacturers - Specialty & Generic,USA,28.82,0.00,0.00,0.00,...,1.81,557162,5.44,0.67,0.64,0.61,-4.54%,2.10,0.00,-1278.57


In [5]:
list(set(data['sector']))

['Consumer Defensive',
 'Basic Materials',
 'Real Estate',
 'Industrials',
 'Communication Services',
 'Energy',
 'Healthcare',
 'Utilities',
 'Technology',
 'Consumer Cyclical',
 'Financial']

In [8]:
import pandas as pd
from yahoo_fin import stock_info as si

In [9]:
# gather stock symbols from major US exchanges
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

# convert DataFrame to list, then to sets
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

# join the 4 sets into one. Because it's a set, there will be no duplicate symbols
symbols = set.union( sym1, sym2, sym3, sym4 )

# Some stocks are 5 characters. Those stocks with the suffixes listed below are not of interest.
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 976 unqualified stock symbols...
There are 10766 qualified stock symbols...


In [10]:
from pandas_datareader import data as pdr

all_symbols = pdr.get_nasdaq_symbols()

all_etfs = all_symbols[all_symbols['ETF'] == True]
all_etfs

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AAA,True,Investment Managers Series Trust II AXS First ...,P,,True,100.0,False,NaN,AAA,AAA,False
AAAU,True,Goldman Sachs Physical Gold ETF Shares,Z,,True,100.0,False,NaN,AAAU,AAAU,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False


In [11]:
len(all_symbols)

11739

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
AAA,True,Investment Managers Series Trust II AXS First ...,P,,True,100.0,False,NaN,AAA,AAA,False
AAAU,True,Goldman Sachs Physical Gold ETF Shares,Z,,True,100.0,False,NaN,AAAU,AAAU,False
AADR,True,AdvisorShares Dorsey Wright ADR ETF,Q,G,True,100.0,False,N,NaN,AADR,False
AAPB,True,GraniteShares 1.75x Long AAPL Daily ETF,Q,G,True,100.0,False,N,NaN,AAPB,False
AAPD,True,Direxion Daily AAPL Bear 1X Shares,Q,G,True,100.0,False,N,NaN,AAPD,False
...,...,...,...,...,...,...,...,...,...,...,...
ZIG,True,The Acquirers Fund,P,,True,100.0,False,NaN,ZIG,ZIG,False
ZROZ,True,PIMCO 25 Year Zero Coupon U.S. Treasury Index...,P,,True,100.0,False,NaN,ZROZ,ZROZ,False
ZSB,True,USCF ETF Trust USCF Sustainable Battery Metals...,P,,True,100.0,False,NaN,ZSB,ZSB,False


In [17]:
all_etfs[all_etfs['Security Name'].str.contains]

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,


In [18]:
import yahoo_fin.stock_info as si

In [21]:
si.get_quote_table(all_etfs.index[0])

{'52 Week Range': '24.03 - 25.03',
 'Ask': '24.53 x 3000',
 'Avg. Volume': 703.0,
 'Beta (5Y Monthly)': 0.0,
 'Bid': '12.25 x 900',
 "Day's Range": '24.51 - 24.51',
 'Expense Ratio (net)': '0.25%',
 'Inception Date': '2020-09-08',
 'NAV': 24.68,
 'Net Assets': '7.33M',
 'Open': 24.59,
 'PE Ratio (TTM)': nan,
 'Previous Close': 24.46,
 'Quote Price': 24.610000610351562,
 'Volume': 50.0,
 'YTD Daily Total Return': '0.99%',
 'Yield': '2.77%'}

In [27]:
# def get_data(ticker, start_date=None, end_date=None, index_as_date=True, interval="1d"):
#     """Downloads historical stock price data into a pandas data frame.  Interval
#     must be "1d", "1wk", "1mo", or "1m" for daily, weekly, monthly, or minute data.
#     Intraday minute data is limited to 7 days.
#     @param: ticker
#     @param: start_date = None
#     @param: end_date = None
#     @param: index_as_date = True
#     @param: interval = "1d"
#     """
#     if interval not in ("1d", "1wk", "1mo", "1m"):
#         raise AssertionError("interval must be of of '1d', '1wk', '1mo', or '1m'")

#     # build and connect to URL
#     site, params = build_url(ticker, start_date, end_date, interval)
#     resp = requests.get(site, params=params)
#     if not resp.ok:
#         raise AssertionError(resp.json())

#     # get JSON response
#     data = resp.json()

#     # get open / high / low / close data
#     frame = pd.DataFrame(data["chart"]["result"][0]["indicators"]["quote"][0])

#     # get the date info
#     temp_time = data["chart"]["result"][0]["timestamp"]
#     if interval != "1m":
#         # add in adjclose
#         frame["adjclose"] = data["chart"]["result"][0]["indicators"]["adjclose"][0][
#             "adjclose"
#         ]
#         frame.index = pd.to_datetime(temp_time, unit="s")
#         frame.index = frame.index.map(lambda dt: dt.floor("d"))
#         frame = frame[["open", "high", "low", "close", "adjclose", "volume"]]
#     else:
#         frame.index = pd.to_datetime(temp_time, unit="s")
#         frame = frame[["open", "high", "low", "close", "volume"]]
#     frame["ticker"] = ticker.upper()
#     if not index_as_date:
#         frame = frame.reset_index()
#         frame.rename(columns={"index": "date"}, inplace=True)
#     return frame


# def get_live_price(ticker):
#     """Gets the live price of input ticker
#     @param: ticker
#     """
#     df = get_data(ticker, end_date=pd.Timestamp.today() + pd.DateOffset(10))
#     return df.close[-1]


def get_quote_table(ticker, dict_result=True):
    """Scrapes data elements found on Yahoo Finance's quote page
    of input ticker
    @param: ticker
    @param: dict_result = True
    """
    site = f"https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}"
    tables = pd.read_html(site)
    data = tables[0].append(tables[1])
    # data.columns = ["attribute", "value"]
    # quote_price = pd.DataFrame(["Quote Price", get_live_price(ticker)]).transpose()
    # quote_price.columns = data.columns.copy()
    # data = data.append(quote_price)
    # data = data.sort_values("attribute")
    # data = data.drop_duplicates().reset_index(drop=True)
    # data["value"] = data.value.map(force_float)
    if dict_result:
        result = {key: val for key, val in zip(data.attribute, data.value)}
        return result
    return data

In [28]:
get_quote_table(all_etfs.index[0])

HTTPError: HTTP Error 404: Not Found

In [ ]:
industry_list = [
    # 'Advertising Agencies',
    # 'Aerospace & Defense',
    # 'Agricultural Inputs',
    # 'Airlines',
    # 'Airports & Air Services',
    # 'Aluminum',
    # 'Apparel Manufacturing',
    # 'Apparel Retail',
    # 'Asset Management',
    # 'Auto & Truck Dealerships',
    # 'Auto Manufacturers',
    # 'Auto Parts',
    # 'Banks - Diversified',
    # 'Banks - Regional',
    # 'Beverages - Brewers',
    # 'Beverages - Non-Alcoholic',
    # 'Beverages - Wineries & Distilleries',
    # 'Biotechnology',
    # 'Broadcasting',
    # 'Building Materials',
    # 'Building Products & Equipment',
    # 'Business Equipment & Supplies',
    # 'Capital Markets',
    # 'Chemicals',
    # 'Closed-End Fund - Debt',
    # 'Closed-End Fund - Equity',
    # 'Closed-End Fund - Foreign',
    # 'Coking Coal',
    # 'Communication Equipment',
    # 'Computer Hardware',
    # 'Confectioners',
    # 'Conglomerates',
    # 'Consulting Services',
    # 'Consumer Electronics',
    # 'Copper',
    # 'Credit Services',
    # 'Department Stores',
    # 'Diagnostics & Research',
    # 'Discount Stores',
    # 'Drug Manufacturers - General',
    # 'Drug Manufacturers - Specialty & Generic',
    # 'Education & Training Services',
    # 'Electrical Equipment & Parts',
    'Electronic Components',
    'Electronic Gaming & Multimedia',
    'Electronics & Computer Distribution',
    'Engineering & Construction',
    'Entertainment',
    'Exchange Traded Fund',
    'Farm & Heavy Construction Machinery',
    'Farm Products',
    'Financial Conglomerates',
    'Financial Data & Stock Exchanges',
    'Food Distribution',
    'Footwear & Accessories',
    'Furnishings, Fixtures & Appliances',
    'Gambling',
    'Gold',
    'Grocery Stores',
    'Health Information Services',
    'Healthcare Plans',
    'Home Improvement Retail',
    'Household & Personal Products',
    'Industrial Distribution',
    'Information Technology Services',
    'Infrastructure Operations',
    'Insurance - Diversified',
    'Insurance - Life',
    'Insurance - Property & Casualty',
    'Insurance - Reinsurance',
    'Insurance - Specialty',
    'Insurance Brokers',
    'Integrated Freight & Logistics',
    'Internet Content & Information',
    'Internet Retail',
    'Leisure',
    'Lodging',
    'Lumber & Wood Production',
    'Luxury Goods',
    'Marine Shipping',
    'Medical Care Facilities',
    'Medical Devices',
    'Medical Distribution',
    'Medical Instruments & Supplies',
    'Metal Fabrication',
    'Mortgage Finance',
    'Oil & Gas Drilling',
    'Oil & Gas E&P',
    'Oil & Gas Equipment & Services',
    'Oil & Gas Integrated',
    'Oil & Gas Midstream',
    'Oil & Gas Refining & Marketing',
    'Other Industrial Metals & Mining',
    'Other Precious Metals & Mining',
    'Packaged Foods',
    'Packaging & Containers',
    'Paper & Paper Products',
    'Personal Services',
    'Pharmaceutical Retailers',
    'Pollution & Treatment Controls',
    'Publishing',
    'REIT - Diversified',
    'REIT - Healthcare Facilities',
    'REIT - Hotel & Motel',
    'REIT - Industrial',
    'REIT - Mortgage',
    'REIT - Office',
    'REIT - Residential',
    'REIT - Retail',
    'REIT - Specialty',
    'Railroads',
    'Real Estate - Development',
    'Real Estate - Diversified',
    'Real Estate Services',
    'Recreational Vehicles',
    'Rental & Leasing Services',
    'Residential Construction',
    'Resorts & Casinos',
    'Restaurants',
    'Scientific & Technical Instruments',
    'Security & Protection Services',
    'Semiconductor Equipment & Materials',
    'Semiconductors',
    'Shell Companies',
    'Silver',
    'Software - Application',
    'Software - Infrastructure',
    'Solar',
    'Specialty Business Services',
    'Specialty Chemicals',
    'Specialty Industrial Machinery',
    'Specialty Retail',
    'Staffing & Employment Services',
    'Steel',
    'Telecom Services',
    'Textile Manufacturing',
    'Thermal Coal',
    'Tobacco',
    'Tools & Accessories',
    'Travel Services',
    'Trucking',
    'Uranium',
    'Utilities - Diversified',
    'Utilities - Independent Power Producers',
    'Utilities - Regulated Electric',
    'Utilities - Regulated Gas',
    'Utilities - Regulated Water',
    'Utilities - Renewable',
    'Waste Management'
]

In [7]:
sorted(list(set(data['industry'])))

['Advertising Agencies',
 'Aerospace & Defense',
 'Agricultural Inputs',
 'Airlines',
 'Airports & Air Services',
 'Aluminum',
 'Apparel Manufacturing',
 'Apparel Retail',
 'Asset Management',
 'Auto & Truck Dealerships',
 'Auto Manufacturers',
 'Auto Parts',
 'Banks - Diversified',
 'Banks - Regional',
 'Beverages - Brewers',
 'Beverages - Non-Alcoholic',
 'Beverages - Wineries & Distilleries',
 'Biotechnology',
 'Broadcasting',
 'Building Materials',
 'Building Products & Equipment',
 'Business Equipment & Supplies',
 'Capital Markets',
 'Chemicals',
 'Closed-End Fund - Debt',
 'Closed-End Fund - Equity',
 'Closed-End Fund - Foreign',
 'Coking Coal',
 'Communication Equipment',
 'Computer Hardware',
 'Confectioners',
 'Conglomerates',
 'Consulting Services',
 'Consumer Electronics',
 'Copper',
 'Credit Services',
 'Department Stores',
 'Diagnostics & Research',
 'Discount Stores',
 'Drug Manufacturers - General',
 'Drug Manufacturers - Specialty & Generic',
 'Education & Training Ser

In [63]:
import yfinance as yf


hist = yf.download(
    tickers=dodds_etf_ticker_list, 
    period='1y', 
    rounding=True, 
    group_by='tickers', 
    auto_adjust=True,
    actions=False,
    show_errors=True
)

for ticker in dodds_etf_ticker_list:
    data = hist[ticker]
    data.columns = [x.lower() for x in data.columns]
    data.index = pd.to_datetime(data.index)
    # data.to_pickle(saveRaw / f"{ticker}.pkl")    

[*********************100%***********************]  86 of 86 completed

1 Failed download:
- FRAK: No data found for this date range, symbol may be delisted


In [64]:
hist

FUE                                SCO                       \
             Open   High    Low  Close Volume   Open   High    Low  Close   
Date                                                                        
2022-02-02  11.11  11.14  10.74  10.99   2200  48.00  49.50  47.90  48.30   
2022-02-03  10.80  10.80  10.80  10.80    100  49.00  49.25  46.50  46.85   
2022-02-04  10.01  10.99  10.01  10.91    900  45.35  45.40  44.40  45.00   
2022-02-07  11.10  11.10  11.10  11.10      0  45.30  45.45  44.65  45.15   
2022-02-08  11.17  11.17  10.78  10.79   1000  46.45  47.60  46.25  46.60   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
2023-01-26  12.01  12.01  12.01  12.01    100  22.54  23.40  22.46  22.99   
2023-01-27  12.17  12.17  12.17  12.17    100  22.54  24.10  22.48  23.93   
2023-01-30  12.25  12.29  12.25  12.29    700  24.46  24.92  23.68  24.85   
2023-01-31  12.36  12.46  12.36  12.46    100  25.00  25.16  23.94  23.98   
2023-02-01  12.38  12.30  12.22  12.30   5367  24.09  25.74  23.96  25.30   

                       DDG                                 CHIU                \
             Volume   Open   High    Low  Close   Volume   Open   High    Low   
Date                                                                            
2022-02-02   677180  12.36  12.44  12.20  12.20   4737.0  17.91  17.91  17.91   
2022-02-03   751240  12.35  12.42  12.34  12.34  15235.0  17.82  17.82  17.82   
2022-02-04  1158780  11.92  12.20  11.92  12.18  32814.0  17.95  17.95  17.95   
2022-02-07   471200  12.28  12.28  12.00  12.02   7093.0  17.96  18.06  17.96   
2022-02-08   682540  12.28  12.32  12.22  12.27  20718.0  18.27  18.27  18.27   
...             ...    ...    ...    ...    ...      ...    ...    ...    ...   
2023-01-26  1136100    NaN    NaN    NaN    NaN      NaN  15.59  15.59  15.59   
2023-01-27  1541900    NaN    NaN    NaN    NaN      NaN  15.62  15.62  15.54   
2023-01-30  1176900    NaN    NaN    NaN    NaN      NaN  15.07  15.14  15.07   
2023-01-31  2082100    NaN    NaN    NaN    NaN      NaN  15.10  15.10  15.08   
2023-02-01  1776135    NaN    NaN    NaN    NaN      NaN  15.05  15.23  15.23   

                           ERTH                               PXJ              \
            Close Volume   Open   High    Low  Close Volume  Open  High   Low   
Date                                                                            
2022-02-02  17.91    100  60.26  60.26  59.24  59.56  15100  3.86  3.86  3.77   
2022-02-03  17.82    100  58.84  59.37  58.45  58.47  14200  3.80  3.80  3.72   
2022-02-04  17.95    100  58.33  59.33  58.25  59.10  23700  3.77  3.89  3.76   
2022-02-07  18.06    300  58.76  59.25  58.57  58.61  13100  3.83  3.92  3.78   
2022-02-08  18.27    100  58.40  59.13  58.21  59.09  61900  3.84  3.85  3.75   
...           ...    ...    ...    ...    ...    ...    ...   ...   ...   ...   
2023-01-26  15.59    100  52.27  52.27  51.34  52.08  33800  5.52  5.53  5.37   
2023-01-27  15.54    300  51.64  53.32  51.58  52.80  18500  5.50  5.54  5.45   
2023-01-30  15.14    300  52.39  52.39  51.82  51.82   8200  5.42  5.49  5.39   
2023-01-31  15.08   1000  51.81  52.77  51.75  52.65  17000  5.41  5.56  5.40   
2023-02-01  15.23    190  52.84  53.94  52.46  53.73  16538  5.57  5.61  5.40   

                            BNO                                   NRGD  \
           Close  Volume   Open   High    Low  Close   Volume     Open   
Date                                                                     
2022-02-02  3.81   74500  24.25  24.29  23.88  24.22  1427500  1825.00   
2022-02-03  3.77   65700  24.06  24.67  23.98  24.60  1073100  1820.00   
2022-02-04  3.87   89000  25.08  25.34  25.08  25.20   990500  1775.00   
2022-02-07  3.87  457600  25.07  25.25  25.03  25.07   841200  1760.00   
2022-02-08  3.79  113100  24.68  24.70  24.32  24.62  1155200  1700.00   
...          ...     ...    ...    ...    ...    ...      ...      ...   

In [65]:
hist['USO'].columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [47]:
from yahooquery import Ticker


# hist_data = Ticker(dodds_etf_ticker_list, asynchronous=True, validate=True, verify=True).history(period='1w')
hist_data = pd.DataFrame(Ticker(dodds_etf_ticker_list).history(period='1y')).reset_index().set_index('date').round(2)

ValueError: Data must be 1-dimensional

In [ ]:
hist_data

{'PSCE':             volume  close      low  open   high  adjclose
 2023-02-01  124213  10.64  10.2723  10.7  10.75     10.64,
 'UCO':                   low      close       open   volume       high   adjclose
 2023-02-01  27.389999  27.879999  29.370001  2981477  29.568001  27.879999,
 'VIS':                   high         low  volume        open       close    adjclose
 2023-02-01  196.199997  191.649994   70296  192.160004  194.729996  194.729996,
 'BNO':                  open  close  volume        low   high  adjclose
 2023-02-01  28.059999   27.4  275386  27.124701  28.15      27.4,
 'DBC':              volume     low       high  close   open  adjclose
 2023-02-01  3077572  24.184  24.700001   24.4  24.68      24.4,
 'NRGD':             volume         low        high       close        open    adjclose
 2023-02-01   86240  287.619995  319.459991  308.829987  287.619995  308.829987,
 'DBE':               low      close   open       high  volume   adjclose
 2023-02-01  21.15  21.280

In [38]:
pd.DataFrame.from_dict(hist_data, orient='index')

AttributeError: 'dict' object has no attribute 'dtype'

In [42]:
pd.DataFrame.from_dict(hist_data)

ValueError: Data must be 1-dimensional

In [43]:
l = ['PSCE', 'UCO', 'VIS', 'BNO', 'DBC', 'NRGD', 'DBE', 'UXI', 'XES', 'ARMR', 'DUSL', 'IMLP', 'EMLP', 'PSCI', 'DBO', 'PDBC', 'PSCU', 'GUNR', 'NANR', 'IYE', 'UGAZF', 'DRIP', 'NRGU', 'IDU', 'IYJ', 'UPW', 'SDP', 'UTSL', 'FTRI', 'UTES', 'FCG', 'BOIL', 'OIH', 'UMI', 'IEO', 'FIDU', 'FUTY', 'RGI', 'SIJ', 'XOP', 'JXI', 'PXE', 'IEZ', 'ERTH', 'FXR', 'GAZ', 'FXU', 'PXJ', 'JHMU', 'CRAK', 'DDG', 'MUNI', 'ERX', 'GNR', 'ERY', 'CHIU', 'UNG', 'XLE', 'UNL', 'ETHO', 'DUG', 'FRAK', 'FUE', 'CHII', 'VPU', 'RLY', 'USOI', 'XLU', 'GUSH', 'DIG', 'FTXN', 'PUI', 'IGE', 'EDOW', 'UGA', 'RYU', 'USL', 'USO', 'HAP', 'EXI', 'JHMI', 'EINC', 'PZD', 'OILK', 'SCO', 'KOLD']

In [44]:
string = ""
for i in l:
    string += (i + " ")
string

'PSCE UCO VIS BNO DBC NRGD DBE UXI XES ARMR DUSL IMLP EMLP PSCI DBO PDBC PSCU GUNR NANR IYE UGAZF DRIP NRGU IDU IYJ UPW SDP UTSL FTRI UTES FCG BOIL OIH UMI IEO FIDU FUTY RGI SIJ XOP JXI PXE IEZ ERTH FXR GAZ FXU PXJ JHMU CRAK DDG MUNI ERX GNR ERY CHIU UNG XLE UNL ETHO DUG FRAK FUE CHII VPU RLY USOI XLU GUSH DIG FTXN PUI IGE EDOW UGA RYU USL USO HAP EXI JHMI EINC PZD OILK SCO KOLD '

In [40]:
hist_data.keys()

dict_keys(['PSCE', 'UCO', 'VIS', 'BNO', 'DBC', 'NRGD', 'DBE', 'UXI', 'XES', 'ARMR', 'DUSL', 'IMLP', 'EMLP', 'PSCI', 'DBO', 'PDBC', 'PSCU', 'GUNR', 'NANR', 'IYE', 'UGAZF', 'DRIP', 'NRGU', 'IDU', 'IYJ', 'UPW', 'SDP', 'UTSL', 'FTRI', 'UTES', 'FCG', 'BOIL', 'OIH', 'UMI', 'IEO', 'FIDU', 'FUTY', 'RGI', 'SIJ', 'XOP', 'JXI', 'PXE', 'IEZ', 'ERTH', 'FXR', 'GAZ', 'FXU', 'PXJ', 'JHMU', 'CRAK', 'DDG', 'MUNI', 'ERX', 'GNR', 'ERY', 'CHIU', 'UNG', 'XLE', 'UNL', 'ETHO', 'DUG', 'FRAK', 'FUE', 'CHII', 'VPU', 'RLY', 'USOI', 'XLU', 'GUSH', 'DIG', 'FTXN', 'PUI', 'IGE', 'EDOW', 'UGA', 'RYU', 'USL', 'USO', 'HAP', 'EXI', 'JHMI', 'EINC', 'PZD', 'OILK', 'SCO', 'KOLD'])

In [41]:
hist_data.values()

dict_values([            volume  close      low  open   high  adjclose
2023-02-01  124213  10.64  10.2723  10.7  10.75     10.64,                   low      close       open   volume       high   adjclose
2023-02-01  27.389999  27.879999  29.370001  2981477  29.568001  27.879999,                   high         low  volume        open       close    adjclose
2023-02-01  196.199997  191.649994   70296  192.160004  194.729996  194.729996,                  open  close  volume        low   high  adjclose
2023-02-01  28.059999   27.4  275386  27.124701  28.15      27.4,              volume     low       high  close   open  adjclose
2023-02-01  3077572  24.184  24.700001   24.4  24.68      24.4,             volume         low        high       close        open    adjclose
2023-02-01   86240  287.619995  319.459991  308.829987  287.619995  308.829987,               low      close   open       high  volume   adjclose
2023-02-01  21.15  21.280001  21.83  21.870001  237606  21.280001,           

In [30]:

# IMPORT ALL THE REQUIRED LIBRARIES
from bs4 import BeautifulSoup as BS
import requests as req
 
url = "https://www.businesstoday.in/latest/economy"
 
webpage = req.get(url)
trav = BS(webpage.content, "html.parser")
M = 1
for link in trav.find_all('a'):
   
    # PASTE THE CLASS TYPE THAT WE GET
    # FROM THE ABOVE CODE IN THIS AND
    # SET THE LIMIT GREATER THAN 35
    if(str(type(link.string)) == "<class 'bs4.element.NavigableString'>"
       and len(link.string) > 35):
 
        print(str(M)+".", link.string)
        M += 1

1.                                 Home
                                
2.                                 Magazine
                                
3.                                 BT TV
                                
4.                                 Market Today
                                
5.                                 Budget 2023
                                
6.                                 Tech Today
                                
7.                                 WEF 2023
                                
8.                                 Industry
                                
9.                                 Money Today
                                
10.                                 Banking
                                
11.                                 Visual Stories
                                
12.                                 Immersives
                                
13.                                 Economy
                 

In [33]:

import yfinance as yahooFinance
 
 
GetFacebookInformation = yahooFinance.Ticker("XLE")
 
# # display Company Sector
# print("Company Sector : ", GetFacebookInformation.info['sector'])
 
# # display Price Earnings Ratio
# print("Price Earnings Ratio : ", GetFacebookInformation.info['trailingPE'])
 
# # display Company Beta
# print(" Company Beta : ", GetFacebookInformation.info['beta'])

In [34]:
GetFacebookInformation.info

{'regularMarketPrice': None, 'preMarketPrice': None, 'logo_url': ''}

In [37]:
from finvizfinance.quote import finvizfinance
import pandas as pd
from pathlib import Path
from datetime import datetime
from os.path import exists


class Recommendations1(object):


    def __init__(self, today):
        self.today = today
        self.saveMonth = str(today)[:7]
        self.saveDay = str(today)[8:10]
        self.reportDate = str(today)[:10]

        self.saveRec = Path(f"data/recommenders/{self.saveMonth}/{self.today}/")
        if not self.saveRec.exists():
            self.saveRec.mkdir(parents=True)

        self.saveRaw = Path(f"data/raw/{self.saveMonth}/{self.today}/")
        if not self.saveRaw.exists():
            self.saveRaw.mkdir(parents=True)

        self.saveScreeners = Path(f"data/screeners/{self.saveMonth}/{self.today}/")
        if not self.saveScreeners.exists():
            self.saveScreeners.mkdir(parents=True)

        self.saveTickers = Path(f"data/tickers/{self.saveMonth}/{self.today}/")
        if not self.saveTickers.exists():
            self.saveTickers.mkdir(parents=True)


    def run_rec1(self, tickers):
        self.ticker_list = tickers
        self.recommendations = []

        for s in self.ticker_list:
            try:
                recommendation = finvizfinance(s).TickerFundament()["Recom"]
            except Exception:
                recommendation = 6.0

            if recommendation == "-":
                recommendation = 6.0

            self.recommendations.append(round(float(recommendation), 2))

        dataframe = pd.DataFrame(
            list(zip(self.ticker_list, self.recommendations)),
            columns=["Company", "Recommendations"],
        ).sort_values("Recommendations")
        dataframe.to_pickle(self.saveRec / "recommender_01A_return_dataFrame.pkl")
        dataframe_02 = dataframe[dataframe["Recommendations"] < 2.6]
        dataframe_02.columns = ["Symbol", "Score"]
        dataframe_02["rank"] = range(1, len(dataframe_02["Symbol"]) + 1)
        dataframe_02 = dataframe_02.set_index("rank")
        # dataframe_02.to_pickle(self.saveRec / "recommender_01_return_dataFrame.pkl")
        return dataframe_02
        

    def run_rec1_personal_port(self, personal_port):
        self.ticker_list = personal_port
        self.recommendations = []

        for s in self.ticker_list:
            try:
                recommendation = finvizfinance(s).TickerFundament()["Recom"]
            except Exception:
                recommendation = 6.0

            if recommendation == "-":
                recommendation = 6.0

            self.recommendations.append(round(float(recommendation), 2))

        dataframe = pd.DataFrame(
            list(zip(self.ticker_list, self.recommendations)),
            columns=["Company", "Recommendations"],
        ).sort_values("Recommendations")
        dataframe_02 = dataframe[dataframe["Recommendations"] < 2.6]
        dataframe_02.columns = ["Symbol", "Score"]
        dataframe_02["rank"] = range(1, len(dataframe_02["Symbol"]) + 1)
        dataframe_02 = dataframe_02.set_index("rank")
        return dataframe, dataframe_02


if __name__ == "__main__":
    Recommendations1('2023-02-01')